In [1]:
import tensorflow as tf

epochs = 50

In [2]:
def mnist_reader(part):
    for i in range(10):
        with open('Part3_{}_{}.csv'.format(i, part)) as f:
            for line in f:
                yield line.strip(), i

def mnist_decoder(csv_line, label):
    FIELD_DEFAULTS = [[0.0]]*(28*28)
    with tf.variable_scope('DataSource'):
        fields = tf.decode_csv(csv_line, FIELD_DEFAULTS)
        im = tf.stack(fields)
        im = tf.reshape(im, (28, 28))
        return im, tf.one_hot(label, depth=10)

In [3]:
tf.reset_default_graph()

with tf.variable_scope('DataSource'):
    dataset = tf.data.Dataset.from_generator(lambda: mnist_reader('Train'),
        # csv_line, label
        (tf.string, tf.int32),
        (tf.TensorShape([]), tf.TensorShape([])
    )).map(mnist_decoder, num_parallel_calls=2) \
      .batch(50) \
      .prefetch(1) \
      .repeat(epochs)
    
    dataset_val = tf.data.Dataset.from_generator(lambda: mnist_reader('Test'),
        (tf.string, tf.int32),
        (tf.TensorShape([]), tf.TensorShape([])
    )).map(mnist_decoder, num_parallel_calls=2) \
      .batch(50) \
      .prefetch(1)

    iter_handle = tf.placeholder(tf.string, shape=[])
    data_iterator = tf.data.Iterator.from_string_handle(iter_handle, dataset.output_types, dataset.output_shapes)
    train_iterator = dataset.make_one_shot_iterator()
    val_iterator = dataset_val.make_initializable_iterator()
    val_init_op = data_iterator.make_initializer(dataset_val)
    images, onehot_labels = data_iterator.get_next()

with tf.variable_scope('Initializer'):
    init_global = tf.global_variables_initializer()
    init_local = tf.local_variables_initializer()

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

progress_epoch = IntProgress(description='epoch', min=0, max=epochs-1)
progress_step = IntProgress(description='step', min=0, max=5000//epochs-1)
progress_valstep = IntProgress(description='val_step', min=0, max=5000//epochs-1)
display(progress_epoch)
display(progress_step)
display(progress_valstep)

config = tf.ConfigProto(
    intra_op_parallelism_threads=2,
    inter_op_parallelism_threads=2,
    allow_soft_placement=True)

with tf.Session(config = config) as sess:
    sess.run(init_global)
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    for epoch in range(epochs):
        progress_epoch.value = epoch
        steps_per_epoch = 5000//epochs
        sess.run(init_local)
        for step in range(steps_per_epoch):
            progress_step.value = step
            sess.run([images, onehot_labels], feed_dict={iter_handle: train_handle})
            
        sess.run([init_local, val_init_op], feed_dict={iter_handle: val_handle})
        for val_step in range(steps_per_epoch):
            progress_valstep.value = val_step
            sess.run([images, onehot_labels], feed_dict={iter_handle: val_handle})


IntProgress(value=0, description='epoch', max=49)

IntProgress(value=0, description='step', max=99)

IntProgress(value=0, description='val_step', max=99)